In [8]:
import pymc3 as pm3
import numpy as np
import numdifftools as ndt
import pandas as pd
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy.optimize import minimize
import matplotlib.pyplot as plt

About Log likelihood estimation : https://rlhick.people.wm.edu/posts/estimating-custom-mle.html

In [88]:
N = 10000
h1 = 4 + np.random.randn(N)
d1 = 5 + np.random.randn(N)

sigma = 3 + .2 * h1

d2 = 3 + 5 * h1 - 2 * d1 + 3 * h1 * d1 + sigma * np.random.randn(N)

df = pd.DataFrame({'d2': d2, 'h1' : h1, 'd1' : d1, 'h1d1': h1 * d1})
df['constant'] = 1

df.head()

,d2,h1,d1,h1d1,constant
0,45.803215,2.421097,5.335857,12.918628,1
1,71.635463,4.064556,4.658930,18.936481,1
2,27.078316,2.082676,3.244444,6.757126,1
3,40.361317,3.336595,2.613230,8.719290,1
4,54.404880,2.761036,5.266261,14.540339,1


In [89]:
def _ll_ols(y, X, beta, gamma):
    mu = X.dot(beta)
    sigma = X[:,:2].dot(gamma)
    return norm(mu,sigma).logpdf(y).sum()    


In [98]:
class MyDepNormML(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(MyDepNormML, self).__init__(endog, exog, **kwds)
    def nloglikeobs(self, params):
        gamma = params[-2:]
        beta = params[:-2]
        ll = _ll_ols(self.endog, self.exog, beta, gamma)
        return -ll
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        self.exog_names.append('gamm1')
        self.exog_names.append('gamm2')
        if start_params == None:
            # Reasonable starting values
            start_params = np.append(np.zeros(self.exog.shape[1]), [0.5,.5])
        return super(MyDepNormML, self).fit(start_params=start_params, 
                                  maxiter=maxiter, maxfun=maxfun, 
                                  **kwds)

In [99]:
sm_ml_manual = MyDepNormML(df.d2,df[['constant','h1', 'd1', 'h1d1']]).fit(start_params = [2, 4, -1, 5, 4, 4])
print(sm_ml_manual.summary())

Optimization terminated successfully.
         Current function value: 2.752625
         Iterations: 359
         Function evaluations: 582
                             MyDepNormML Results                              
Dep. Variable:                     d2   Log-Likelihood:                -27526.
Model:                    MyDepNormML   AIC:                         5.506e+04
Method:            Maximum Likelihood   BIC:                         5.509e+04
Date:                Thu, 02 Dec 2021                                         
Time:                        15:05:51                                         
No. Observations:               10000                                         
Df Residuals:                    9996                                         
Df Model:                           3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [92]:
# the fitted coefficients:
sm_ols_manual.params


array([-1.03133392, -0.87559337, -1.05305212,  4.08276043,  2.54928339,
        1.03135554])